In [2]:
import torch
print(torch.version)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

<module 'torch.version' from '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/version.py'>
True
4
0
NVIDIA A10G


In [3]:
!git clone https://github.com/arthurcornelio88/stable-audio-tools-sam.git

Cloning into 'stable-audio-tools-sam'...
remote: Enumerating objects: 1345, done.
remote: Counting objects: 100% (820/820), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 1345 (delta 663), reused 654 (delta 570), pack-reused 525
Receiving objects: 100% (1345/1345), 67.48 MiB | 39.99 MiB/s, done.
Resolving deltas: 100% (902/902), done.


In [4]:
!pip install stable-audio-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 89.9 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
INFO: pip is looking at multiple versions of librosa 

In [5]:
import os
os.environ["WANDB_API_KEY"] = "de922f7f8b9e172867a482fbc22d32acfb02042f"

In [6]:
!wandb login

wandb: Currently logged in as: arthur-cornelio (sammusic). Use `wandb login --relogin` to force relogin


In [7]:
!conda install -y -c conda-forge libsndfile

Retrieving notices: ...working... done
done
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::python==3.10.14=hd12c33a_0_cpython
  - conda-forge/noarch::attrs==23.2.0=pyh71513ae_0
  - conda-forge/linux-64::brotli-python==1.1.0=py310hc6cd4ac_1
  - conda-forge/noarch::cached_property==1.5.2=pyha770c72_1
  - conda-forge/noarch::certifi==2024.2.2=pyhd8ed1ab_0
  - conda-forge/noarch::charset-normalizer==3.3.2=pyhd8ed1ab_0
  - conda-forge/noarch::colorama==0.4.6=pyhd8ed1ab_0
  - conda-forge/noarch::cycler==0.12.1=pyhd8ed1ab_0
  - conda-forge/linux-64::debugpy==1.8.1=py310hc6cd4ac_0
  - conda-forge/noarch::decorator==5.1.1=pyhd8ed1ab_0
  - conda-forge/noarch::defusedxml==0.7.1=pyhd8ed1ab_0
  - conda-forge/linux-64::docutils==0.16=py310hff52083_4
  - conda-forge/noarch::entrypoints==0.4=pyhd8ed1ab_0
  - conda-forge/noarch::exceptiongroup==1.2.0=pyhd8ed1ab_2
  - conda-

In [8]:
!pip install soundfile

In [9]:
!pip install torch torchvision

In [10]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 71.0 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.1-cp310-cp310-linux_x86_64.whl size=198377647 sha256=cf6e577925a9b80015a4f94a82a464baf13165dfd3c20289a11d14f8d181f274
  Stored in directory: /home/ec2-user/.cache/pip/wheels/91/6a/38/f0faa036b4ac73a73247386f1ab1bb4cb4f6e72e6861a779f1
Successfully built flash-attn


In [11]:
import boto3

s3 = boto3.resource('s3')
bucket_name = 'sagemaker-eu-west-2-339713155623' #Arthur's sagemaker bucket
tar_file_key = 'webdataset_audio/audio_dataset-00001.tar'  # The key (path) to your tar archive in S3 (Arthur's)

try:
    obj = s3.Object(bucket_name, tar_file_key).get()
    print("Successfully accessed tar archive!")
except Exception as e:
    print(f"Error accessing tar archive: {e}")

Successfully accessed tar archive!


In [21]:
#télécharge le model en local
import boto3

s3 = boto3.client('s3')
s3.download_file('sagemaker-eu-west-2-339713155623', 'model.safetensors', 'model.safetensors')

KeyboardInterrupt: 

In [22]:
import shutil

# Move the model
shutil.move('./model.safetensors', './stable-audio-tools-sam/stable_open_model_files/model.safetensors')

# Move custom_metadata.py to the root (necessary for dataset.py find it)
shutil.move('./stable-audio-tools-sam/custom_metadata.py', './custom_metadata.py')

FileNotFoundError: [Errno 2] No such file or directory: './model.safetensors'

In [23]:
#installe les bonnes version pour torch et torch audio
!pip install torch torchaudio

In [24]:
!nvidia-smi

Mon Jul 15 20:25:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1B.0 Off |                    0 |
|  0%   24C    P8               9W / 300W |      3MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
import torch
torch.cuda.set_per_process_memory_fraction(0.9)

In [26]:
import torch
torch.backends.cuda.max_split_size_mb = 128

In [27]:
!pip install -U deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.9 MB/s eta 0:00:00
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Created wheel for deepspeed: filename=deepspeed-0.14.4-py3-none-any.whl size=1445517 sha256=04f98316a02ddc332f691d29e4fb175031453f756cbd9ba7c6da30aa94d03340
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8e/bc/a3/608e90bbb301848b78fd75d24d6d43ba3074de968fc0e397ac
Successfully built deepspeed


In [ ]:
# We need to upload manually 'dataset_config.json', model_config.json' and custom_metadataetc'
# Modify, in dataset_config, the s3_path to Arthur's

In [28]:
!pip install setuptools==65.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1


In [ ]:
!python ./train.py --model-config ./stable_open_model_files/model_config.json --dataset-config ./stable_open_model_files/dataset_config.json --name arthur-training --save-dir ./checkpoints --pretrained-ckpt-path ./stable_open_model_files/model.safetensors --batch-size 16 --num-gpus 4 --strategy deepspeed

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
wandb: Currently logged in as: arthur-cornelio (sammusic). Use `wandb login --relogin` to force relogin


In [36]:
print(torchaudio.__file__)

NameError: name 'torchaudio' is not defined

In [ ]:
import wandb
api = wandb.Api()

# run is specified by <entity>/<project>/<run_id>
run = api.run("sammusic/arthur-training/peach-totem-3")

# save the metrics for the run to a csv file
metrics_dataframe = run.history()
metrics_dataframe.to_csv("metrics.csv")

In [ ]:
import wandb
api = wandb.Api()

run = api.run("sammusic/arthur-training/peach-totem-3")
if run.state == "finished":
    for i, row in run.history().iterrows():
      print(row["_timestamp"], row["accuracy"])